Этап 1. Загрузка и предобработка данных

In [1]:
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from torchvision.transforms.v2 import RandomHorizontalFlip, RandomVerticalFlip, RandomRotation
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split, Dataset, DataLoader


class TransformDataset(Dataset):
    def __init__(self, dataset, transforms):
        super(TransformDataset, self).__init__()
        self.dataset = dataset
        self.transforms = transforms

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        x, y = self.dataset[idx]
        return self.transforms(x), y


def get_dataloaders(path_to_train_dataset, path_to_test_dataset, batch_size):
    dataset = ImageFolder(path_to_train_dataset)
    test_dataset = ImageFolder(path_to_test_dataset)
    train_dataset, val_dataset = random_split(dataset, [0.8, 0.2])

    # Трансформации для датасетов
    train_transforms = Compose([
        RandomHorizontalFlip(p=0.2),
        RandomVerticalFlip(p=0.2),
        RandomRotation([-5, 5], fill=255.),
        Resize((224, 224)),
        ToTensor(),
        Normalize((0.5), (0.5))
    ])

    test_transforms = Compose([
        Resize((224, 224)),
        ToTensor(),
        Normalize((0.5), (0.5))
    ])

    # Добавление трансформаций к датасетам
    train_dataset = TransformDataset(train_dataset, train_transforms)
    val_dataset = TransformDataset(val_dataset, test_transforms)
    test_dataset = TransformDataset(test_dataset, test_transforms)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Проверка
    print("Количество изображений в train:", len(train_dataset))
    print("Количество изображений в val:", len(val_dataset))
    print("Список классов:", dataset.classes)
    return train_loader, val_loader, test_loader, dataset.classes

In [2]:
# Создаем даталоадеры
batch_size = 256
path_to_train_dataset = "./dataset/ogyeiv2/train"
path_to_test_dataset = "./dataset/ogyeiv2/test"
train_loader, val_loader, test_loader, all_classes = get_dataloaders(path_to_train_dataset,
                                                                     path_to_test_dataset,
                                                                     batch_size)

Количество изображений в train: 1882
Количество изображений в val: 470
Список классов: ['acc_long_600_mg', 'advil_ultra_forte', 'akineton_2_mg', 'algoflex_forte_dolo_400_mg', 'algoflex_rapid_400_mg', 'algopyrin_500_mg', 'ambroxol_egis_30_mg', 'apranax_550_mg', 'aspirin_ultra_500_mg', 'atoris_20_mg', 'atorvastatin_teva_20_mg', 'betaloc_50_mg', 'bila_git', 'c_vitamin_teva_500_mg', 'calci_kid', 'cataflam_50_mg', 'cataflam_dolo_25_mg', 'cetirizin_10_mg', 'cold_fx', 'coldrex', 'concor_10_mg', 'concor_5_mg', 'condrosulf_800_mg', 'controloc_20_mg', 'covercard_plus_10_mg_2_5_mg_5_mg', 'coverex_4_mg', 'diclopram_75-mg_20-mg', 'dorithricin_mentol', 'dulsevia_60_mg', 'enterol_250_mg', 'favipiravir_meditop_200_mg', 'ibumax_400_mg', 'jutavit_c_vitamin', 'jutavit_cink', 'kalcium_magnezium_cink', 'kalium_r', 'koleszterin_kontroll', 'lactamed', 'lactiv_plus', 'laresin_10_mg', 'letrox_50_mikrogramm', 'lordestin_5_mg', 'merckformin_xr_1000_mg', 'meridian', 'metothyrin_10_mg', 'mezym_forte_10_000_egyseg'

Этап 2. Объявление модели

In [3]:
import torch.nn as nn
import torchvision.models as models


class PharmDetector(nn.Module):
    def __init__(self, num_classes, freeze_backbone=True):
        super(PharmDetector, self).__init__()
        # Используем предобученную ResNet50
        self.backbone = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
        # Замораживаем часть слоев для трансферного обучения
        if freeze_backbone:
            # Замораживаем все слои кроме последних двух
            for name, param in self.backbone.named_parameters():
                if not name.startswith('layer4') and not name.startswith('fc'):
                    param.requires_grad = False
        # Заменяем последний полносвязный слой под наше количество классов
        in_features = self.backbone.fc.in_features
        self.backbone.fc = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(in_features, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        return self.backbone(x)


In [4]:
# Создаем модель
model = PharmDetector(num_classes=len(all_classes), freeze_backbone=True)

Этап 3. Обучение или дообучение

In [5]:
import numpy as np
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix


# Вывод метрик
def print_model_metrics(model, dataloader, device, class_names):
    model.eval()
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, predictions = torch.max(outputs, 1)

            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(
        all_labels, all_predictions, average='weighted', zero_division=0
    )
    if accuracy > 0.75:
        print(f"Общая точность (accuracy): {accuracy:.2%}")
        print(f"Точность (precision): {precision:.2%}")
        print(f"Полнота (recall): {recall:.2%}")
        print(f"Cреднее между точностью и полнотой (F1-Score): {f1:.2%}")

        cm = confusion_matrix(all_labels, all_predictions)
        errors_per_class = [cm[i].sum() - cm[i, i] for i in range(len(class_names))]

        # Топ-5 классов с ошибками
        error_indices = np.argsort(errors_per_class)[-5:][::-1]
        print("\nТоп-5 классов с ошибками:")
        for idx in error_indices:
            if errors_per_class[idx] > 0:
                print(f"  {class_names[idx]}: {errors_per_class[idx]} ошибок")

        # Классы без ошибок
        perfect_classes = [class_names[i] for i in range(len(class_names))
                           if errors_per_class[i] == 0 and cm[i].sum() > 0]
        if perfect_classes:
            print(f"\nКлассы без ошибок: {', '.join(perfect_classes)}")

In [6]:
import torch
from src.metrics import print_model_metrics


# Код обучения одной эпохи
def train_one_epoch(epoch_index, model, optimizer, criterion, train_loader, device):
    running_loss = 0.
    total_samples = 0.
    for batch_index, data in enumerate(train_loader):
        # Извлечение батча
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        # Обнуление градиентов
        optimizer.zero_grad()
        # Прямое распространение
        outputs = model(inputs)
        # Подсчёт ошибки
        loss = criterion(outputs, labels)
        # Обратное распространение
        loss.backward()
        # Обновление весов
        optimizer.step()
        # Суммирование ошибки
        running_loss += loss.item() * inputs.size(0)
        total_samples += inputs.size(0)
    # Возвращаем среднюю ошибку за всю эпоху
    avg_epoch_loss = running_loss / total_samples
    return avg_epoch_loss


# Цикл обучения
def run_train(model, criterion, optimizer, train_loader, val_loader, test_loader, class_names, epochs, device, best_vloss = 1e5):
    for epoch in range(epochs):
        print(f'Эпоха {epoch}')
        # Перевод модели в режим обучения
        model.train(True)
        # Эпоха обучения
        avg_loss = train_one_epoch(epoch, model, optimizer, criterion, train_loader, device)
        # Перевод модели в режим валидации
        model.eval()
        running_vloss = 0.0
        total_vsamples = 0
        # Валидация
        with torch.no_grad():
            for i, vdata in enumerate(val_loader):
                vinputs, vlabels = vdata
                vinputs, vlabels = vinputs.to(device), vlabels.to(device)
                voutputs = model(vinputs)
                vloss = criterion(voutputs, vlabels)
                running_vloss += vloss.item() * vinputs.size(0)
                total_vsamples += vinputs.size(0)

        avg_vloss = running_vloss / total_vsamples
        # Сохранение лучшей модели
        if avg_vloss < best_vloss:
            best_vloss = avg_vloss
            model_path = f'meds_classifier_{epoch}.pt'
            torch.save(model.state_dict(), model_path)

        print(f'В конце эпохи ошибка train {avg_loss:.4f}, ошибка val {avg_vloss:.4f}')
        print_model_metrics(model, test_loader, device, class_names)


In [7]:
import torch
from torch import nn, optim


# Настройка гиперпараметров
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f'Обучаем на {device}')

# Запуск обучения с тестированием и выводом метрик
run_train(model=model, criterion=criterion, optimizer=optimizer, train_loader=train_loader,
          val_loader=val_loader, test_loader=test_loader, class_names=all_classes, epochs=10, device=device)

Обучаем на cuda
Эпоха 0
В конце эпохи ошибка train 4.0392, ошибка val 4.3945
Эпоха 1
В конце эпохи ошибка train 2.4597, ошибка val 3.6326
Эпоха 2
В конце эпохи ошибка train 1.4365, ошибка val 3.2716
Эпоха 3
В конце эпохи ошибка train 0.9484, ошибка val 3.3307
Эпоха 4
В конце эпохи ошибка train 0.6709, ошибка val 4.4143
Эпоха 5
В конце эпохи ошибка train 0.4797, ошибка val 2.0202
Эпоха 6
В конце эпохи ошибка train 0.3552, ошибка val 0.7761
Эпоха 7
В конце эпохи ошибка train 0.2898, ошибка val 0.6126
Общая точность (accuracy): 82.54%
Точность (precision): 83.06%
Полнота (recall): 82.54%
Cреднее между точностью и полнотой (F1-Score): 80.44%

Топ-5 классов с ошибками:
  ambroxol_egis_30_mg: 6 ошибок
  meridian: 6 ошибок
  normodipine_5_mg: 6 ошибок
  naturland_d_vitamin_forte: 6 ошибок
  theospirex_150_mg: 5 ошибок

Классы без ошибок: acc_long_600_mg, advil_ultra_forte, algoflex_forte_dolo_400_mg, algoflex_rapid_400_mg, algopyrin_500_mg, apranax_550_mg, aspirin_ultra_500_mg, atoris_20_mg, 

Оцените полученные метрики, ответив на вопросы:
1. На каких 5 классах модель ошибается чаще всего?
2. Почему модель может ошибаться на этих классах?
3. На каких классах модель не совершает ошибок?
4. Почему эти классы модель распознаёт безошибочно?
5. Как можно улучшить точность классификатора?
6. Как ещё можно проанализировать результаты и ошибки модели?

Ответы:
1. При точности выше 75% выводятся классы ошибок.
2. В этих классах все таблетки однотонные белые. Есть обшие формы (круглые, овальные и пилюли).
3. При точности выше 75% выводятся классы без ошибок.
4. Из-за уникальности вида таблеток, болше уникальный признаков (цвета или конкретный цвет, формы).
5. Увеличивая датасет исходных. Увеличивая свойств аугментации. Более тяжелую исходную модель и больше колочества эпох обучения. 
6. Визуализация промежуточных слоев обработки. Вывод изображения - результатов работы свертки для наглядной проверки работы.